In [ ]:
import sys
sys.path.append("..")

Fine tune a pretrained DistilBert model on dataset (`conll2003`) for named entity recognition. This will be our basic starting model.

In [2]:
from src.models.ModelTrainer import ModelTrainer
from src.features.build_features import load_token_class_dataset

dataset = load_token_class_dataset()
model_trainer = ModelTrainer()

Found cached dataset conll2003 (/Users/michellelo/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98)


  0%|          | 0/3 [00:00<?, ?it/s]

Found cached dataset conll2003 (/Users/michellelo/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98)


  0%|          | 0/3 [00:00<?, ?it/s]

Found cached dataset conll2003 (/Users/michellelo/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /Users/michellelo/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98/cache-93721f4e51ca03c2.arrow
Loading cached processed dataset at /Users/michellelo/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98/cache-6c55135baeeb1675.arrow
Loading cached processed dataset at /Users/michellelo/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98/cache-bd590531a89b56a6.arrow


In [3]:
from src.models.evaluate_model import evaluate
from src.visualization.get_models import get_basic_model

# Train and evaluate a basic model on pretrained DistilBert
basic_model = get_basic_model(model_trainer)
evaluate(basic_model, dataset["validation"])

Loading saved basic model...


Loading cached processed dataset at /Users/michellelo/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98/cache-e420dfe9bc0558f3.arrow


{'LOC': {'precision': 0.9568034557235421, 'recall': 0.9646162221012521, 'f1': 0.9606939550013553, 'number': 1837}, 'MISC': {'precision': 0.8824786324786325, 'recall': 0.89587852494577, 'f1': 0.8891280947255111, 'number': 922}, 'ORG': {'precision': 0.914179104477612, 'recall': 0.9134973900074571, 'f1': 0.9138381201044385, 'number': 1341}, 'PER': {'precision': 0.9624060150375939, 'recall': 0.9728555917480999, 'f1': 0.9676025917926566, 'number': 1842}, 'overall_precision': 0.9373956594323873, 'overall_recall': 0.9449680242342645, 'overall_f1': 0.9411666107945021, 'overall_accuracy': 0.9895642693041549, 'total_time_in_seconds': 228.96883300697664, 'samples_per_second': 14.194071556896013, 'latency_in_seconds': 0.07045194861753128}


Identify concept neurons in the basic model associated with location names.

In [4]:
from src import basic_model_path, basic_activations_path
from src.visualization.ModelAnalyzer import ModelAnalyzer

# Identify neurons in the basic model to ablate
basic_analyser = ModelAnalyzer(basic_model_path, basic_activations_path)

Loading json activations from ../data/interim/basic_activations.json...
2500 7.0
Number of tokens:  74052
length of source dictionary:  13561
length of target dictionary:  2
74052
Total instances: 74052
['Kardashian', 'reducing', 'lawn', 'Bennett', 'Re', 'Christie', 'Dave', 'Clermont', 'review', 'seized', 'suspension', 'Ryu', 'Ruled', 'violation', 'colored', 'Klum', '97', 'minute', 'fail', 'Marceno']
Number of samples:  74052
Stats: Labels with their frequencies in the final set
N/A 70750
SEM:named_entity:location 3302
Training classification probe
Creating model...
Number of training instances: 74052
Number of classes: 2


epoch [1/10]: 0it [00:00, ?it/s]

Epoch: [1/10], Loss: 0.0024


epoch [2/10]: 0it [00:00, ?it/s]

Epoch: [2/10], Loss: 0.0020


...

Epoch: [9/10], Loss: 0.0013


epoch [10/10]: 0it [00:00, ?it/s]

Epoch: [10/10], Loss: 0.0013


Evaluating: 0it [00:00, ?it/s]

Score (accuracy) of the probe: 0.99
{'__OVERALL__': 0.9901690703829741, 'N/A': 0.996, 'SEM:named_entity:location': 0.8652331920048455}


In [5]:
neurons_to_prune = basic_analyser.identify_concept_neurons()

Correlation matrix size (#neurons x #neurons): (5376, 5376)
Number of clusters detected: 1582
[2949, 3717, 4485, 4300, 4353, 4584, 4654, 4679, 4812, 5040, 5068, 5121, 5220, 5292, 5352, 3857, 4084, 4475, 4852, 418, 1186, 3631, 4399, 417, 1185, 1953, 2721, 3489, 4257, 4252, 5020, 2620, 3388, 4156, 4544, 904, 1672, 2440, 3208, 3329, 3842, 3848, 3895, 3963, 3972, 3976, 3978, 3980, 3988, 3993, 4036, 4040, 4043, 4057, 4085, 4097, 4107, 4122, 4134, 4172, 4193, 4195, 4209, 4223, 4273, 4282, 4305, 4311, 4343, 4349, 4351, 4397, 4441, 4459, 4462, 4468, 4525, 4533, 4576, 4609, 4610, 4615, 4616, 4622, 4623, 4625, 4627, 4628, 4629, 4635, 4639, 4640, 4645, 4656, 4664, 4666, 4669, 4673, 4677, 4682, 4687, 4690, 4695, 4696, 4698, 4699, 4703, 4705, 4707, 4708, 4711, 4712, 4725, 4731, 4732, 4733, 4735, 4737, 4739, 4740, 4747, 4748, 4750, 4760, 4761, 4768, 4772, 4774, 4775, 4781, 4782, 4792, 4797, 4804, 4808, 4819, 4820, 4825, 4826, 4834, 4838, 4844, 4853, 4860, 4862, 4865, 4867, 4869, 4870, 4875, 4880, 48

In [7]:
basic_top_words = basic_analyser.show_top_words(neurons_to_prune)

In [11]:
import pandas as pd
prune_df = pd.DataFrame(data=basic_top_words.items(),columns=['neuron_id', 'top_words'])
prune_df.to_csv('../data/processed/locations_prune.csv')
prune_df

,neuron_id,top_words
0,2949,"[(Pan, 1.0), (Latin, 0.9383950441874997), (fre..."
1,3717,"[(Fe, 1.0), (Lumpur, 0.9844285162937593), (Pen..."
2,4485,"[(Gardens, 1.0), (Pan, 0.9798361918673336), (A..."
3,4300,"[(Adobe, 1.0), (Brazil, 0.9731878205152977), (..."
4,4353,"[(Spurs, 1.0), (Google, 0.9663719793668513), (..."
...,...,...
5370,522,"[(Buffalo, 1.0), (stops, 0.8810935090075411), ..."
5371,1290,"[(stops, 1.0), (household, 0.8549134427482343)..."
5372,2058,"[(stops, 1.0), (backing, 0.8402476614178218), ..."
5373,2826,"[(Ankara, 1.0), (Oslo, 0.9704224418915807), (N..."


Prune the identified concept neurons from the model by setting their weights to zero.

In [12]:
from src import basic_model_path, pruned_model_path
from src.models.prune_model import prune_model
from src.models.evaluate_model import evaluate
from src.visualization.get_models import get_pruned_model

# If the model has been saved
pruned_model = get_pruned_model(pruned_model_path, model_trainer)

# Ablate neurons
# pruned_model = prune_model(basic_model_path, model_trainer, neurons_to_prune)
# pruned_model.save_pretrained(pruned_model_path)
evaluate(pruned_model, dataset["validation"])

Loading cached processed dataset at /Users/michellelo/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98/cache-e420dfe9bc0558f3.arrow
/Users/michellelo/Desktop/research/neuron-identification-1/venv/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/michellelo/Desktop/research/neuron-identification-1/venv/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'LOC': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1837}, 'MISC': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 922}, 'ORG': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1341}, 'PER': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1842}, 'overall_precision': 0.0, 'overall_recall': 0.0, 'overall_f1': 0.0, 'overall_accuracy': 0.8325026284023208, 'total_time_in_seconds': 486.06120557902614, 'samples_per_second': 6.686400730394435, 'latency_in_seconds': 0.14955729402431575}


Retrain the pruned model until performance recovers. Examine:

- In which neurons does the concept of location names reappear in the pruned model?
- Is there any relation between the new concept (location names) and what concepts originally existed in these neurons?

In [ ]:
from src import pruned_model_path, retrained_model_path
from src.visualization.get_models import get_retrained_model

# Retrain the pruned model
retrained_model = get_retrained_model(retrained_model_path, pruned_model_path, model_trainer)

In [ ]:
from src import retrained_model_path, retrained_activations_path
from src.visualization.ModelAnalyzer import ModelAnalyzer

# Examine the retrained model for concepts and compare with old model
retrained_analyser = ModelAnalyzer(retrained_model_path, retrained_activations_path)

In [ ]:
new_concept_neurons = retrained_analyser.identify_concept_neurons()

In [ ]:
post_top_words = retrained_analyser.show_top_words(new_concept_neurons)
pre_top_words = basic_analyser.show_top_words(new_concept_neurons)

In [ ]:
plastic_df = pd.DataFrame(data=[pre_top_words.items(), post_top_words.items()],columns=['neuron_id', 'pre_top_words', 'post_top_words'])
plastic_df.to_csv('../data/processed/locations_plastic.csv')
plastic_df